In [ ]:
import numpy as np
import pandas as pd
import param
import panel as pn
import holoviews as hv
import hvplot.pandas
pn.extension()

In [ ]:
df_cap = pd.read_csv('../results/capacity.csv')
df_fuel = pd.read_csv('../results/fuel_cons.csv')
df_ELsup = pd.read_csv('../results/Elsup.csv')
df_emissions = pd.read_csv('../results/emissions.csv')
df_trade = pd.read_csv('../results/trade.csv')
df_investments = pd.read_csv('../results/investments.csv')

In [ ]:
df_investments['value'] = np.divide(df_investments['value'],1000)

In [ ]:
names = {'bah':'Bahrain',
         'kuw':'Kuwait',
         'ksa':'Saudi Arabia',
         'omn':'Oman',
         'qat':'Qatar',
         'uae':'UAE'}

df_cap = df_cap.replace(names)
df_fuel = df_fuel.replace(names)
df_ELsup = df_ELsup.replace(names)
df_emissions = df_emissions.replace(names)
df_trade = df_trade.replace(names)
df_investments = df_investments.replace(names)

In [ ]:
countries = ['Bahrain','Kuwait','Oman','Qatar','Saudi Arabia','UAE']
fuels = ['HFO','diesel','NG','Arablight']
scenarios = ['A','B','C','D','E','F','G','H']

In [ ]:
df_trade.head(2)

In [ ]:
class EMISSIONS1(param.Parameterized):
    
# EMISSIONS SUBPLOTS
# https://hvplot.holoviz.org/user_guide/Subplots.html
    
    #country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    show_legend = param.Boolean(False, doc="Show the legend")

#    def get_data_emissions(self):
#        start_date = self.year_range[0]
#        end_date = self.year_range[1]
#        country = self.country_selector
#        
#        mask = (df_emissions['c'] == country) & (df_emissions['trun'] >= start_date) & (df_emissions['trun'] <= end_date)
#        _df = df_emissions.loc[mask]
#        
#        #df_pivot = _df.pivot_table(index=['trun','scenario'],columns='c',values='value',fill_value=0)
#        #df_pivot = df_pivot.reset_index(drop=False)
#        
#        return _df
#    
#    def plot_emission(self,scenario):
#        country = self.country_selector
#        _df  = self.get_data_emissions()
#        
#        mask = (_df['scenario']==scenario)
#        data = _df.loc[mask].reset_index(drop=True)
#        
#        title = 'Emissions for ' + country
#        
#        plot = (data.hvplot(
#            x='trun',
#            y='value',
#            #by='scenario',
#            subplots=False,
#            value_label='million tons',
#            alpha=0.7,
#            width=300,
#            shared_axes=True,
#            rot=90) * data.hvplot.scatter(
#            x='trun',
#            y='value',
#            by='scenario',
#            subplots=False,
#            title=title,
#            line_color='k',
#            shared_axes=True))
#        return plot
#    
#    def plot_emissions(self):
#        p_list = []
#        for scenario in scenarios:
#            p = self.plot_emission(scenario)
#            p_list.append(p)
#            layout_emissions = hv.Layout(p_list)
#        return layout_emissions
    
    def get_data_emissions(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        #country = self.country_selector
        
        mask = (df_emissions['trun'] >= start_date) & (df_emissions['trun'] <= end_date)
        _df = df_emissions.loc[mask]
        
        #df_pivot = _df.pivot_table(index=['trun','scenario'],columns='c',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        return _df
    
    def plot_emissions_scenario(self,scenario):
        _df  = self.get_data_emissions()
        mask = (_df['scenario']==scenario)
        _df = _df.loc[mask].reset_index(drop=True)
        
        df_pivot = _df.pivot_table(index=['trun','scenario'],columns='c',values='value',fill_value=0)
        data = df_pivot.reset_index(drop=False)
        c_list = data.drop(['scenario','trun'],axis=1).columns.values.tolist()
        title = 'Emissions for ' + scenario
        
        plot = data.hvplot.bar(
            x='trun',
            y=c_list,
            title=title,
            value_label='million tons',
            alpha=0.7,
            width=300,
            stacked=True,
            shared_axes=True,
            legend=self.show_legend,
            rot=90)
        return plot
    
    def plot_emissions_country(self,country):
        _df  = self.get_data_emissions()
        mask = (_df['c']==country)
        _df = _df.loc[mask].reset_index(drop=True)
        
        df_pivot = _df.pivot_table(index=['trun','c'],columns='scenario',values='value',fill_value=0)
        data = df_pivot.reset_index(drop=False)
        s_list = data.drop(['c','trun'],axis=1).columns.values.tolist()
        title = 'Emissions for ' + country
        
        plot = (data.hvplot(
            x='trun',
            y=s_list,
            title=title,
            value_label='million tons',
            alpha=0.7,
            width=300,
            shared_axes=False,
            legend=self.show_legend,
            rot=90) * data.hvplot.scatter(
            x='trun',
            y=s_list,
            title=title,
            line_color='k',
            legend=self.show_legend,
            shared_axes=False))
        return plot
    
    def plot_emissions_s(self):
        p_list = []
        for scenario in scenarios:
            p = self.plot_emissions_scenario(scenario)
            p_list.append(p)
            layout_emissions = hv.Layout(p_list)
        return layout_emissions
    
    def plot_emissions_c(self):
        p_list = []
        for country in countries:
            p = self.plot_emissions_country(country)
            p_list.append(p)
            layout_emissions = hv.Layout(p_list)
        return layout_emissions    

em1 = EMISSIONS1()

In [ ]:
class EMISSIONS2(param.Parameterized):
    
# EMISSIONS SUBPLOTS
# https://hvplot.holoviz.org/user_guide/Subplots.html
    
    #country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    show_legend = param.Boolean(False, doc="Show the legend")
    
    def get_data_emissions(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        
        mask = (df_emissions['trun'] >= start_date) & (df_emissions['trun'] <= end_date)
        _df = df_emissions.loc[mask]    
        return _df
       
    def plot_emissions_country(self,country):
        _df  = self.get_data_emissions()
        mask = (_df['c']==country)
        _df = _df.loc[mask].reset_index(drop=True)
        
        df_pivot = _df.pivot_table(index=['trun','c'],columns='scenario',values='value',fill_value=0)
        data = df_pivot.reset_index(drop=False)
        s_list = data.drop(['c','trun'],axis=1).columns.values.tolist()
        title = 'Emissions for ' + country
        
        plot = (data.hvplot(
            x='trun',
            y=s_list,
            title=title,
            value_label='million tons',
            alpha=0.7,
            width=300,
            shared_axes=False,
            legend=self.show_legend,
            rot=90,
            ylim=(0,500)) * data.hvplot.scatter(
            x='trun',
            y=s_list,
            title=title,
            line_color='k',
            legend=self.show_legend,
            shared_axes=False,
            ylim=(0,500)))
        return plot
    
    def plot_emissions_c(self):
        p_list = []
        for country in countries:
            p = self.plot_emissions_country(country)
            p_list.append(p)
            layout_emissions = hv.Layout(p_list)
        return layout_emissions    

em2 = EMISSIONS2()

In [ ]:
class FUEL(param.Parameterized):
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    #scenario_selector = param.ObjectSelector(default="B", objects=scenarios)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    show_legend = param.Boolean(False, doc="Show the legend")
    
    def get_data_fuel(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        country = self.country_selector
        
        mask = (df_fuel['c'] == country) & (df_fuel['trun'] >= start_date) & (df_fuel['trun'] <= end_date)
        
        _df = df_fuel.loc[mask]
        
        #df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        return _df
    
    def plot_fuel(self,scenario):    
        #scenario = self.scenario_selector
        _df  = self.get_data_fuel()
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        
        mask = (df_pivot['scenario']==scenario)               # come back to this
        data = df_pivot.loc[mask].reset_index(drop=True)      
        
        title = self.country_selector + ' Scenario ' + scenario
        
        fuel_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(
            x='trun',
            y=fuel_list,
            #by='scenario',
            stacked=True,
            subplots=False,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90)
        return plot
    
    def plot_fuels(self):
        p_list = []
        for scenario in scenarios:
            p = self.plot_fuel(scenario)
            p_list.append(p)
            layout_fuel = hv.Layout(p_list)
        return layout_fuel
    
fuel = FUEL()

In [ ]:
class ELSUP(param.Parameterized):
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    show_legend = param.Boolean(False, doc="Show the legend")
    
    def get_data_sup(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        country = self.country_selector
        
        mask =  (df_ELsup['c'] == country) & (df_ELsup['trun'] >= start_date) & (df_ELsup['trun'] <= end_date)
        
        _df = df_ELsup.loc[mask]
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='ELp',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot

    def plot_sup(self,scenario):
        #country = self.country_selector
        _df  = self.get_data_sup()
        
        #df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='ELp',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        mask = (_df['scenario']==scenario)
        data = _df.loc[mask].reset_index(drop=True)      

        title = self.country_selector + ' for Scenario ' + scenario
        
        ELp_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(
            x='trun',
            y=ELp_list,
            value_label='GW',
            title=title,
            alpha=0.4,
            stacked=True,
            width=300,
            shared_axes=False,
            legend=self.show_legend,
            rot=90)
        return plot

    def plot_sups(self):
        p_list = []
        for scenario in scenarios:
            p = self.plot_sup(scenario)
            p_list.append(p)
            layout_sup = hv.Layout(p_list)
        return layout_sup
    
sup=ELSUP()

In [ ]:
class CAPACITY(param.Parameterized):
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    #scenario_selector = param.ObjectSelector(default="G", objects=scenarios)
    show_legend = param.Boolean(False, doc="Show the legend")
    
    def get_data_cap(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        country = self.country_selector
        
        mask =  (df_cap['c'] == country) & (df_cap['trun'] >= start_date) & (df_cap['trun'] <= end_date)
        
        _df = df_cap.loc[mask]
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='ELp',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
   
    def plot_cap(self,scenario):
        country = self.country_selector
        #scenario = self.scenario_selector
        _df  = self.get_data_cap()
        mask = (_df['scenario']==scenario)
        data = _df.loc[mask].reset_index(drop=True)      
        
        title = country + ' for Scenario ' + scenario
        
        ELp_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(x='trun',
                               y=ELp_list,
                               value_label='GW',
                               title=title,
                               alpha=0.4,
                               stacked=True,
                               width=300,
                               shared_axes=False,
                               legend=self.show_legend,
                               rot=90)
        return plot
        
    def plot_caps(self):
        p_list = []
        for scenario in scenarios:
            p = self.plot_cap(scenario)
            p_list.append(p)
            layout_cap = hv.Layout(p_list)
        return layout_cap
    
cap = CAPACITY()

In [ ]:
class INVESTMENTS(param.Parameterized):
    
    def get_data(self):
        _df = df_investments
        
        _df = (_df
               .drop(['trun'],axis=1)
               .pivot_table(index=['scenario'],
                            columns=['tech'],
                            fill_value=0,
                            aggfunc=np.sum)
              )
        
        _df.columns = _df.columns.droplevel()
        return _df

    def plot_data(self):
        data = self.get_data()
        #_df  = self.get_data()
             
        title = 'Cumulative investments'
        ELp_list = data.columns.values.tolist()

        plot = data.hvplot.bar(value_label='billion USD',
                               title=title,
                               alpha=0.4,
                               stacked=True,
                               width=800,
                               height=600)
        return plot
    
inv = INVESTMENTS()

In [ ]:
#dashboard = pn.Tabs(('At a glance',
#                     pn.Column(
#                     pn.Row(g.param),
#                     pn.Row(g.plot_cap,g.plot_fuel),
#                     pn.Row(g.plot_el,g.plot_emissions))
#))

about_capacities = """\
The capacity of power plants and water desalination plants change over time based on investment decisions made by the model.

"""

about_fuel = """\
Fuel consumption.

"""

about_sup = """\
Electricity produced by power plants.

"""

about_emissions = """\
CO2 emissions.

"""

about_inv = """\
Cumulative investments by all countries.\
Add:
- investments by country
- investments by technology

"""

desc_capacity = pn.layout.Row(pn.pane.Markdown(about_capacities))
desc_fuel = pn.layout.Row(pn.pane.Markdown(about_fuel))
desc_sup = pn.layout.Row(pn.pane.Markdown(about_sup))
desc_em = pn.layout.Row(pn.pane.Markdown(about_emissions))
desc_inv = pn.layout.Row(pn.pane.Markdown(about_inv))


widgets_cap = pn.WidgetBox('### Select your query',cap.param)
widgets_fuel = pn.WidgetBox('### Select your query',fuel.param)
widgets_sup = pn.WidgetBox('### Select your query',sup.param)
widgets_em1 = pn.WidgetBox('### Select your query',em1.param)
widgets_em2 = pn.WidgetBox('### Select your query',em2.param)

dashboard = pn.Tabs(('Capacity',
    pn.Row(
        pn.Column(
            pn.Row('## Capacities'),
            pn.Row(widgets_cap),
            pn.Row(desc_capacity)),
        pn.Column(
            pn.Row(cap.plot_caps))                 
            )))

dashboard.append(('Fuel',
    pn.Row(
        pn.Column(
            pn.Row(widgets_fuel),
            pn.Row(desc_fuel)),
        pn.Column(
            pn.Row(fuel.plot_fuels))
                 )))

dashboard.append(('Electricity',
    pn.Row(
        pn.Column(
            pn.Row('## Electricity supply'),
            pn.Row(widgets_sup),
            pn.Row(desc_sup)),
        pn.Column(
            pn.Row(sup.plot_sups))                      
            )))

dashboard.append(('Emissions by Scenario',
    pn.Row(
        pn.Column(
            pn.Row('## CO2 emissions'),
            pn.Row(widgets_em1),
            pn.Row(desc_em)),
        pn.Column(
            pn.Row(em1.plot_emissions_s))                  
            )))

dashboard.append(('Emissions by Country',
    pn.Row(
        pn.Column(
            pn.Row('## CO2 emissions'),
            pn.Row(widgets_em2),
            pn.Row(desc_em)),
        pn.Column(
            pn.Row(em2.plot_emissions_c))                  
            )))

#dashboard.append(('Exchange',
#             pn.Column(
#                 pn.Row('## Electricity exchange'),
#                 pn.Row(ex.param),
#                 pn.Row(ex.plot_C,ex.table_C),
#                 pn.Row(ex.plot_D,ex.table_D),
#                 pn.Row(ex.plot_G,ex.table_G),
#                 pn.Row(ex.plot_H,ex.table_H))
#             ))
#
dashboard.append(('Investment',
                 pn.Column(
                     pn.Row(desc_inv),
                     pn.Row(inv.plot_data))
                 ))

#dashboard.append(('Side by side',
#                 pn.Column(pn.Row(c.param.country_selector1,c.param.scenario_selector,c.param.show_legend),
#                 pn.Row(c.param.country_selector2,c.param.year_range),
#                 pn.Row('## Side by side country comparisons'),
#                 pn.Row(c.cap_plot1,c.cap_plot2),
#                 pn.Row(c.fuel_plot1,c.fuel_plot2),
#                 pn.Row(c.sup_plot1,c.sup_plot2))
#                 ))

In [ ]:
dashboard.show()

In [ ]:
class EXCHANGE(GENERAL):
    def get_data_exchange(self):
        _df = df_trade
        _df = df_trade.groupby(['scenario','c','cc']).sum().drop('trun',axis=1)
        _df = _df.reset_index().drop('scenario',axis=1)
        display(_df)
        plot = hv.Chord(_df)
        return plot

In [ ]:
J = EXCHANGE()
J.get_data_exchange()

In [ ]:
#! panel serve --show --port 5000 dashboard.ipynb